In [1]:
from sys import maxsize
import numpy as np
import matplotlib.pyplot as plt
import perfplot
import numba as nb
from numba.experimental import jitclass
from math import floor, sqrt, ceil
from time import time

In [16]:
@jitclass([("V", nb.int64[:]), ("size", nb.int64)])
class Parte(object):
    def __init__(self, V, size):
        self.V = V
        self.size = size

    def __setitem__(self, index, new_value):
        self.V[index] = new_value

    def __getitem__(self, index):
        return self.V[index]

    def pop(self):
        self.size -= 1
        return self.V[self.size]

#### Métodos quadráticos de ordenação

In [3]:
@nb.jit
def insertion_sort(parte: Parte):
    for j in range(1, parte.size):
        key = parte.V[j]
        i = j - 1
        while (i >= 0) and (parte.V[i] > key):
            parte.V[i + 1] = parte.V[i]
            i -= 1
            parte.V[i + 1] = key
    return parte

#### Heap

#### Square Root Sort

In [41]:
# VAZIO = -maxsize
VAZIO = -1


@nb.njit
def make_max_heap(V: np.ndarray):
    """Transforma o array em um max-heap"""
    for i in range(V.size // 2, -1, -1):
        heapify_down(V, i)


@nb.njit
def heapify_down(heap, i):
    # Mapeia o heap para uma binary tree
    maior = i
    filho_esq = (i * 2) + 1
    filho_dir = (i * 2) + 2

    if (filho_esq < heap.size) and heap[filho_esq] > heap[maior]:
        maior = filho_esq

    if (filho_dir < heap.size) and (heap[filho_dir] > heap[maior]):
        maior = filho_dir

    if maior != i:
        heap[i], heap[maior] = heap[maior], heap[i]  # troca
        heapify_down(heap, maior)

@nb.njit
def heapify_up(V, i):
    pai = (i - 1) // 2

    if (pai >= 0) and (V[i] > V[pai]):
        V[pai], V[i] = V[i], V[pai]  # troca
        heapify_up(V, pai)

@nb.njit
def heap_insert(heap:np.ndarray, e):
    heap[-1] = e
    
    heapify_up(heap, heap.size - 1)

@nb.njit
def heap_pop(heap: Parte) -> int:
    maior = heap[0]
    ultimo = heap.pop()

    if heap.size != 0:
        heap[0] = ultimo
        heapify_down(heap, 0)

    return maior

@nb.njit
def heap_sort(parte: Parte):
    make_max_heap(parte.V)
    size = parte.size

    for i in range(size):
        maior = heap_pop(parte)
        parte[size - 1 - i] = maior

    parte.size = size

    return parte

@nb.njit
def particiona_array(array):
    n = array.size

    max_len_parte = floor(sqrt(n))
    n_partes = ceil(n / max_len_parte)
    resto = n % max_len_parte

    partes = []
    for i in range(n_partes):
        start = i * max_len_parte
        nxt = (i + 1) * max_len_parte

        if (i != n_partes - 1) or (resto == 0):
            p = Parte(array[start:nxt], max_len_parte)
        else:
            filler = [VAZIO] * (max_len_parte - resto)
            final_array = np.append(array[start:n], filler)

            p = Parte(final_array, resto)

        partes.append(p)

    return partes

@nb.njit
def sqrt_sort_quadratico(V: np.ndarray):
    n = V.size

    partes = [insertion_sort(parte) for parte in particiona_array(V)]

    solucao = np.zeros(n, dtype=np.int64)
    aux_max_list = np.full(len(partes), VAZIO)

    for iteracao in range(n):
        # Preenche o vetor aux_max_list com os maiores valores de cada parte disponível
        for i, parte in enumerate(partes):
            if (aux_max_list[i] == VAZIO) and (parte.size > 0):
                aux_max_list[i] = parte[parte.size - 1]

        # Pega o index do maior elemento de "aux_max_list"
        max_index = np.argmax(aux_max_list)

        solucao[iteracao] = aux_max_list[max_index]

        aux_max_list[max_index] = VAZIO  # indica q a vaga está disponível de novo

        partes[max_index].size -= 1  # remove o elemento da parte

    return solucao[::-1]

@nb.njit
def sqrt_sort_heap(V: np.ndarray):
    n = V.size

    partes = [heap_sort(parte) for parte in particiona_array(V)]

    solucao = np.zeros(n, dtype=np.int64)
    aux_max_list = np.full(len(partes), VAZIO)

    for iteracao in range(n):
        # Preenche o vetor aux_max_list com os maiores valores de cada parte disponível
        for i, parte in enumerate(partes):
            if (aux_max_list[i] == VAZIO) and (parte.size > 0):
                aux_max_list[i] = parte[parte.size - 1]

        # Pega o index do maior elemento de "aux_max_list"
        max_index = np.argmax(aux_max_list)

        solucao[iteracao] = aux_max_list[max_index]

        aux_max_list[max_index] = VAZIO  # indica q a vaga está disponível de novo

        partes[max_index].size -= 1  # remove o elemento da parte

    return solucao[::-1]


runtimes = []
# for size in sizes:
for _ in range(1):
    np.random.seed(124)  # Seed the random number generator
    array = np.random.randint(1000, size=10**1)

    start = time()

    # res = sqrt_sort_quadratico(array)
    res = sqrt_sort_heap(array)
    # print(res)
    print(np.array_equal(sorted(array), res))

    total = time() - start
    runtimes.append(total)

    print("{:.3f}".format(total))

print("\nMean:", round(np.mean(runtimes), 3))


True
2.852

Mean: 2.852


In [5]:

# plt.loglog(sizes, runtimes, label="Tempo")
# plt.show()